In [1]:
# Import libraries

import sys
import logging
import pandas as pd
import numpy as np
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.weather import load_weather
from library.network import build_network

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 500
load_target = 15 #TWh

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [4]:
# Build the network

network = build_network(index, resolution, geography, target_load, assumptions, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

generator_capacity = model.variables["Generator-p_nom"]
link_capacity = model.variables["Link-p_nom"]

## Add offwind constraint
if False:
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
lhs = link_capacity.loc["battery-discharge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 833.20it/s]
INFO:linopy.io: Writing time: 0.72s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [4e-06, 3e+02]
  Cost   [8e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 3e+03]
Presolving model
45264 rows, 39432 cols, 115352 nonzeros  0s
34549 rows, 28717 cols, 110504 nonzeros  0s
33919 rows, 28087 cols, 111112 nonzeros  0s

Solving MIP model with:
   33919 rows
   28087 cols (0 binary, 5 integer, 0 implied int., 28082 continuous)
   111112 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.4s
 S       0       0         0   0.00%   -inf            26665133008.65     Large        0      0      0         0    19.4s
 R       0       0         

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 46739 primals, 105141 duals
Objective: 2.65e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [6]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop       2185.191365                 0.0  5.411536e+05   
          biogas          500.000000                 0.0  8.570971e+05   
          onwind         7058.571429                 0.0  1.076153e+07   
          solar          5640.269460                 0.0  5.570286e+06   
Link      AC             2665.343120                 0.0  1.283381e+07   
          biogas          500.000000                 0.0  8.570971e+05   
          h2             3300.000000                 0.0  3.947954e+06   
          mixedgas       2100.000000                 0.0  1.625037e+06   
Load      -                 0.000000                 0.0  0.000000e+00   
Store     h2            35018.352432                 0.0  1.973977e+06   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator backstop  0.000000e+00  5.411536e+05  0.000000e+00         0.028270   
          biogas    0.000000e+00  8.570971e+05  0.000000e+00         0.195684   
          onwind    0.000000e+00  1.076153e+07  0.000000e+00         0.174042   
          solar     0.000000e+00  5.570286e+06  0.000000e+00         0.112739   
Link      AC        1.283381e+07  0.000000e+00  1.283381e+07         0.549665   
          biogas    8.570971e+05  0.000000e+00  8.570971e+05         0.195684   
          h2        5.471987e+06 -1.524033e+06  0.000000e+00         0.189290   
          mixedgas  2.831074e+06 -1.206038e+06  2.831074e+06         0.153896   
Load      -         1.500000e+07 -1.500000e+07  0.000000e+00              NaN   
Store     h2        1.973977e+06  1.469743e-09  0.000000e+00         0.442162   

                     Curtailment  Capital Expenditure  \
Generator backstop  0.000000e+00         0.000000e+00   
          biogas    0.000000e+00         0.000000e+00   
          onwind    1.290452e+07         9.514676e+09   
          solar     8.013656e+05         2.929702e+09   
Link      AC        0.000000e+00         0.000000e+00   
          biogas    0.000000e+00         0.000000e+00   
          h2        0.000000e+00         1.309340e+09   
          mixedgas  0.000000e+00         2.796449e+09   
Load      -         0.000000e+00         0.000000e+00   
Store     h2        0.000000e+00         2.669694e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             5.688607e+09      0.0           NaN  
          biogas               1.095147e+09      0.0           NaN  
          onwind               3.097124e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          biogas               0.000000e+00      0.0           NaN  
          h2                   0.000000e+00      0.0           NaN  
          mixedgas             1.821952e+08      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     h2                   0.000000e+00      0.0           NaN